<a href="https://colab.research.google.com/github/sinitsky96/DDBM-projA/blob/main/part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [337]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [338]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import *
import re
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import col,from_json
import json



In [339]:
spark=SparkSession.builder.appName('Part1').getOrCreate()


In [340]:
schema = StructType([
      StructField("movie_id",IntegerType(),True), \
      StructField("genres",StringType(),True) ,\
     StructField("overview",StringType(),True) ,\
     StructField("production_companies",StringType(),True), \
     StructField("production_countries",StringType(),True), \
     StructField("release_date",StringType(),True), \
      StructField("revenue",IntegerType(),True) ,\
      StructField("spoken_languages",StringType(),True) ,\
      StructField("tagline",StringType(),True), \
     StructField("title",StringType(),True), \
     StructField("cities",StringType(),True), \
     ])




In [341]:
df_movies=spark.read.format("csv")\
  .option("mode","DROPMALFORMED")\
  .option("delimiter", ",")\
  .option("header","true")\
  .schema(schema)\
    .load("/content/drive/MyDrive/Files for DDBM/projA/movies.csv")
df_movies.show()
df_movies.count()

+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+--------------------+
|movie_id|              genres|            overview|production_companies|production_countries|release_date|  revenue|    spoken_languages|             tagline|               title|              cities|
+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+--------------------+
|     862|[{'id': 16, 'name...|Led by Woody, And...|[{'name': 'Pixar ...|[{'iso_3166_1': '...|  30/10/1995|373554033|[{'iso_639_1': 'e...|                null|           Toy Story|['Eilat', 'Tel Av...|
|    8844|[{'id': 12, 'name...|When siblings Jud...|[{'name': 'TriSta...|[{'iso_3166_1': '...|  15/12/1995|262797249|[{'iso_639_1': 'e...|Roll the dice and...|             Jumanji|['Jerusalem'

45566

In [342]:
split_cols = pyspark.sql.functions.split(df_movies['cities'], ',')
df1 = df_movies.withColumn('city1', split_cols.getItem(0)) \
    .withColumn('city2', split_cols.getItem(1)) \
    .withColumn('city3', split_cols.getItem(2))\
        .withColumn('city4', split_cols.getItem(3))\
                .withColumn('city5', split_cols.getItem(4))


df1.show()

+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+--------------------+------------+------------+-------------+------------+-------------+
|movie_id|              genres|            overview|production_companies|production_countries|release_date|  revenue|    spoken_languages|             tagline|               title|              cities|       city1|       city2|        city3|       city4|        city5|
+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+--------------------+------------+------------+-------------+------------+-------------+
|     862|[{'id': 16, 'name...|Led by Woody, And...|[{'name': 'Pixar ...|[{'iso_3166_1': '...|  30/10/1995|373554033|[{'iso_639_1': 'e...|                null|           Toy Story|['Eilat', 'Te

In [343]:
from pyspark.sql.types import NullType
def cleanCities(city):
  if city==None:
    return None
  flag=False
  cities=["Tel Aviv","Eilat","Tiberius","Jerusalem","Haifa"]
  for c in cities:
    if c in city:
      city=c
      flag = True
  if flag==False:
    city=None
  return city
CityUDF=udf(lambda z:cleanCities(z),StringType())

In [344]:
for i in range(1,6):
 df1= df1.withColumn(f"City{i}", CityUDF(f"City{i}"))
df1=df1.drop("cities")
df1.show()



+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+
|movie_id|              genres|            overview|production_companies|production_countries|release_date|  revenue|    spoken_languages|             tagline|               title|    City1|    City2|    City3|    City4|    City5|
+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+
|     862|[{'id': 16, 'name...|Led by Woody, And...|[{'name': 'Pixar ...|[{'iso_3166_1': '...|  30/10/1995|373554033|[{'iso_639_1': 'e...|                null|           Toy Story|    Eilat| Tel Aviv|Jerusalem|    Haifa|     null|
|    8844|[{'id': 12, 'name...|When siblings Jud...|[{'name': 'TriSta...|[{'

In [345]:
def jformat(cell):
  tokens=cell.split("}")
  result=""
  for t in tokens:
    search = re.search("'name': '(.*)'", t)
    if search!= None:
      result+=","+search.group(1)
  return(result[1:])
def numberOfComma(cell):
  c= cell.count(",")
  return c+1
  
jUDF=udf(lambda z:jformat(z),StringType())
cUDF=udf(lambda z:numberOfComma(z),IntegerType())

df2= df1.withColumn("genres", jUDF("genres"))
df2.show()
df3 = df2.withColumn("genrees", cUDF("genres"))
df3.sort(df3.genrees.desc())



+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+
|movie_id|              genres|            overview|production_companies|production_countries|release_date|  revenue|    spoken_languages|             tagline|               title|    City1|    City2|    City3|    City4|    City5|
+--------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+
|     862|Animation,Comedy,...|Led by Woody, And...|[{'name': 'Pixar ...|[{'iso_3166_1': '...|  30/10/1995|373554033|[{'iso_639_1': 'e...|                null|           Toy Story|    Eilat| Tel Aviv|Jerusalem|    Haifa|     null|
|    8844|Adventure,Fantasy...|When siblings Jud...|[{'name': 'TriSta...|[{'

DataFrame[movie_id: int, genres: string, overview: string, production_companies: string, production_countries: string, release_date: string, revenue: int, spoken_languages: string, tagline: string, title: string, City1: string, City2: string, City3: string, City4: string, City5: string, genrees: int]

In [349]:
split_cols = pyspark.sql.functions.split(df2['genres'], ',')
df3 = df2.withColumn('genre1', split_cols.getItem(0)) \
              .withColumn('genre2', split_cols.getItem(1)) \
              .withColumn('genre3', split_cols.getItem(2))\
              .withColumn('genre4', split_cols.getItem(3))\
              .withColumn('genre5', split_cols.getItem(4))\
              .withColumn('genre6', split_cols.getItem(5))\
              .withColumn('genre7', split_cols.getItem(6))\
              .withColumn('genre8', split_cols.getItem(7))\
 




df3=df3.drop("genres")
df3.show()

+--------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+---------+---------+---------+--------+------+------+------+------+
|movie_id|            overview|production_companies|production_countries|release_date|  revenue|    spoken_languages|             tagline|               title|    City1|    City2|    City3|    City4|    City5|   genre1|   genre2|   genre3|  genre4|genre5|genre6|genre7|genre8|
+--------+--------------------+--------------------+--------------------+------------+---------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+---------+---------+---------+--------+------+------+------+------+
|     862|Led by Woody, And...|[{'name': 'Pixar ...|[{'iso_3166_1': '...|  30/10/1995|373554033|[{'iso_639_1': 'e...|                null|           Toy Story|    Eilat|

In [361]:
def cleanstrng(cell):
  c=cell.replace("[","")
  c=c.replace("]","")
  c=c.replace("'", '"')
  return c
cleanUDF=udf(lambda z:cleanstrng(z),StringType())
df4 = df3.withColumn("spoken_languages", cleanUDF("spoken_languages"))



df4=df4.select('movie_id','overview',json_tuple(df4.spoken_languages,'iso_639_1')).show()

+--------+--------------------+----+
|movie_id|            overview|  c0|
+--------+--------------------+----+
|     862|Led by Woody, And...|  en|
|    8844|When siblings Jud...|  en|
|   15602|A family wedding ...|  en|
|   31357|"Cheated on, mist...|null|
|   11862|Just when George ...|  en|
|     949|Obsessive master ...|  en|
|   11860|An ugly duckling ...|  fr|
|   45325|A mischievous you...|  en|
|    9091|International act...|  en|
|     710|James Bond must u...|  en|
|    9087|Widowed U.S. pres...|  en|
|   12110|When a lawyer sho...|  en|
|   21032|An outcast half-w...|  en|
|   10858|An all-star cast ...|  en|
|    1408|Morgan Adams and ...|  en|
|     524|The life of the g...|  en|
|    4584|Rich Mr. Dashwood...|  en|
|       5|It's Ted the Bell...|  en|
|    9273|Summoned from an ...|  en|
|   11517|A vengeful New Yo...|  en|
+--------+--------------------+----+
only showing top 20 rows

